<a href="https://colab.research.google.com/github/njokinjuguna/Machine-learning-Models/blob/main/CancerPredictiveAnalysisInKenya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Path to the KNH workbook
knh_workbook_path = 'KNH_Data.xlsx'  # Replace with the actual path

# Load the workbook
knh_data = pd.ExcelFile(knh_workbook_path)

# Loop through each sheet in the workbook and display its structure and content
for sheet_name in knh_data.sheet_names:
    print(f"\nSheet name: {sheet_name}")

    # Load the sheet into a DataFrame
    sheet_data = knh_data.parse(sheet_name)

    # Display basic information about the sheet
    print("Structure:")
    print(sheet_data.info())

    # Display the first few rows of the sheet
    print("First few rows:")
    print(sheet_data.head())

# Optional: Close the workbook if necessary
knh_data.close()



Sheet name: Original Database 
Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 56 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Questionnaire No                                             500 non-null    object 
 1   Site                                                         500 non-null    object 
 2   Age                                                          500 non-null    int64  
 3   Sex                                                          500 non-null    object 
 4   Residence                                                    500 non-null    object 
 5   Referral                                                     500 non-null    object 
 6   Referred From                                                331 non-null    object 
 7   Gastric cancer                       

In [2]:
import pandas as pd

# Path to the MTRH workbook
mtrh_workbook_path = 'MTRH_Data.xlsx'  # Replace with the actual path

# Load the workbook
mtrh_data = pd.ExcelFile(mtrh_workbook_path)

# Loop through each sheet in the workbook and display its structure and content
for sheet_name in mtrh_data.sheet_names:
    print(f"\nSheet name: {sheet_name}")

    # Load the sheet into a DataFrame
    sheet_data = mtrh_data.parse(sheet_name)

    # Display basic information about the sheet
    print("Structure:")
    print(sheet_data.info())

    # Display the first few rows of the sheet
    print("First few rows:")
    print(sheet_data.head())

# Optional: Close the workbook if necessary
knh_data.close()


Sheet name: Original Database
Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 57 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Questionnaire No                                             500 non-null    object 
 1   Site                                                         500 non-null    object 
 2   Age                                                          500 non-null    int64  
 3   Sex                                                          500 non-null    object 
 4   Residence                                                    500 non-null    object 
 5   Referral                                                     496 non-null    object 
 6   Referred From                                                207 non-null    object 
 7   Gastric cancer                        

**merging the 2 data sets**

In [6]:
import pandas as pd

# Paths to the two files
file1_path = 'KNH_Data.xlsx'  # Replace with the actual path to the KNH file
file2_path = 'MTRH_Data.xlsx'  # Replace with the actual path to the MTRH file

# Load each file's 'Cleaned DB' sheet into DataFrames
df1 = pd.read_excel(file1_path, sheet_name='Cleaned DB')
df2 = pd.read_excel(file2_path, sheet_name='Cleaned DB')


# Standardizing column names in KNH to match MTRH
column_names_mtrh = df2.columns.tolist()  # Getting column names from MTRH
df1.columns = column_names_mtrh  # Applying these column names to KNH

# Concatenate the two DataFrames after standardizing column names
combined_df = pd.concat([df1, df2], ignore_index=True)

# Strip any leading or trailing whitespace in column names
combined_df.columns = combined_df.columns.str.strip()

# Rename the last five columns explicitly to 2008 through 2012
combined_df.columns.values[-5:] = [2008, 2009, 2010, 2011, 2012]



# Rename "Questionnaire No" to "PatientID"
combined_df.rename(columns={"Questionnaire No": "PatientID"}, inplace=True)

# Make "PatientID" unique by appending the "Hospital" name
combined_df["PatientID"] = combined_df["PatientID"].astype(str) + "_" + combined_df["Site"]

# Rename "Site" to "Hospital"
combined_df.rename(columns={"Site": "Hospital"}, inplace=True)

# Load the 'Coding' sheet from the KNH workbook to get the region mappings
coding_df = pd.read_excel(file1_path, sheet_name='Coding')

# Extract the regions and their codes from 'Unnamed: 14'
# We assume the codes are 1 to 7, and '0' for "others/out of kenya"
region_codes = list(range(8))
region_names = coding_df['Unnamed: 14'].iloc[1:9].values  # Adjusted to extract the relevant rows

# Create a dictionary to map Province codes to region names
province_mapping = dict(zip(region_codes, region_names))

# Map the 'Province' column in the combined DataFrame using the province_mapping
combined_df['Province'] = combined_df['Province'].map(province_mapping)

# Ensure "Breast cancer" column exists
if 'Breast cancer' in combined_df.columns:
    # Update records for MTRH (filter by 'Hospital' == 'MTRH')
    mtrh_mask = combined_df['Hospital'] == 'MTRH'
    combined_df.loc[mtrh_mask, 'Breast cancer'] = combined_df.loc[mtrh_mask, 'Breast cancer'].apply(
        lambda x: 0 if pd.isna(x) or str(x).strip() == "" else 1  # Blank/NaN = 0, any non-blank = 1
    )
    print("Updated 'Breast cancer' column for MTRH records.")
else:
    print("'Breast cancer' column not found in the DataFrame.")

# # Print all column names to verify the actual name of the 'Breast cancer' column
# print("Column names in the DataFrame:", combined_df.columns.tolist())

# Define the range of cancer columns to process
cancer_columns = combined_df.loc[:, 'Gastric cancer':'Multiple Myeloma'].columns

# Replace NaN with 0 and 'yes'/'Yes' (case-insensitive) with 1 in cancer columns
for column in cancer_columns:
    combined_df[column] = combined_df[column].fillna(0)  # Replace NaN with 0
    combined_df[column] = combined_df[column].apply(lambda x: 1 if x != 0 else 0)  # Replace any non-0 with 1

# Clean the 'OTHERS' column (column 'AN')
combined_df['OTHERS'] = combined_df['OTHERS'].fillna(0)  # Replace NaN with 0
combined_df[column] = combined_df[column].apply(lambda x: 1 if x != 0 else 0)  # Replace any non-0 with 1

# Define the range of cancer checking method columns (AP-AY)
method_columns = combined_df.loc[:, 'Method_Biopsy':'Mammogram'].columns

# Replace NaN with 0 and all other values with 1 in method columns
for column in method_columns:
    combined_df[column] = combined_df[column].fillna(0)  # Replace NaN with 0
    combined_df[column] = combined_df[column].apply(lambda x: 1 if x != 0 else 0)  # Replace any non-0 with 1

# Clean the 'others' column (AZ)
combined_df['others'] = combined_df['others'].fillna(0)  # Replace NaN with 0
combined_df['others'] = combined_df['others'].apply(lambda x: 1 if x != 0 else 0)  # Replace any non-0 with 1


# # Print the combined DataFrame to verify the changes
# print(combined_df.head())

# Save the combined DataFrame to a new Excel file
output_path = 'Test_semiFinale.xlsx'
combined_df.to_excel(output_path, index=False)
print(f"Combined dataset with standardized columns saved to {output_path}")


Updated 'Breast cancer' column for MTRH records.


<ipython-input-6-24cb710740ee>:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df[column] = combined_df[column].fillna(0)  # Replace NaN with 0


Combined dataset with standardized columns saved to Test_semiFinale.xlsx
